# Import Libraries

In [1]:
from swarmintelligence import EelGrouperOptimizerCopy as EelGrouperOptimizer
from swarmintelligence import Utilization
import os
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import time # calculate CPU time
import pickle
import itertools

# import package metrics
from sewar.full_ref import *

# Load Data Testing

In [2]:
# load the model from disk
test_dataset_misc = pickle.load(open('datasets/test_dataset.pkl', 'rb'))
gray_misc_dataset = test_dataset_misc['gray']
rgb_misc_dataset = test_dataset_misc['rgb']
# gray_misc_dataset = dict(itertools.islice(gray_misc_dataset.items(), 2))
rgb_misc_dataset = dict(itertools.islice(rgb_misc_dataset.items(), 2))

In [3]:
rgb_misc_dataset

{'AirplaneF16.tiff': array([[[181,   0,  98],
         [185, 206, 202],
         [165, 207, 197],
         ...,
         [150, 166, 206],
         [150, 164, 202],
         [131, 147, 196]],
 
        [[141,   0, 108],
         [199, 193, 204],
         [195, 189, 200],
         ...,
         [162, 182, 204],
         [141, 155, 195],
         [119, 139, 197]],
 
        [[141,   0, 108],
         [197, 196, 203],
         [193, 193, 199],
         ...,
         [174, 195, 209],
         [144, 159, 196],
         [116, 144, 193]],
 
        ...,
 
        [[160,   0, 172],
         [210, 215, 212],
         [211, 214, 214],
         ...,
         [181, 185, 191],
         [173, 175, 190],
         [158, 144, 168]],
 
        [[163,   0, 173],
         [210, 216, 213],
         [210, 215, 215],
         ...,
         [168, 163, 177],
         [184, 182, 190],
         [167, 164, 184]],
 
        [[ 35, 127,  32],
         [ 33, 123,  34],
         [ 34, 125,  38],
         ...,
        

# Load Optimal Parameter

In [4]:
# optimal_params = pickle.load(open('results/hyperparameter_tuning/memGWO/optimal_params.pkl', 'rb'))
# optimal_params

In [5]:
optimal_params = {
    'eelSize':30,
    'maxIteration': 10
}

# Evaluation

In [6]:
thresholds = [2,3,4,5]
fitness_functions = ['otsu']
objs = ['max']
# n_runs = 30
n_runs = 1

In [7]:
gwo_evaluation_results = {
    'image_name': [],
    'thresholds': [],
    'fitness_function': [],
    'obj':[],
    'Mean best_thresholds R':[],
    'Mean best_thresholds G':[],
    'Mean best_thresholds B':[],
    'Mean Fitness R':[],
    'Mean Fitness G':[],
    'Mean Fitness B':[],
    'Mean CPU_time (seconds)': [],
    'Mean MSE':[],
    'Mean RMSE':[],
    'Mean PSNR':[],
    'Mean SSIM':[],
    'Mean UQI':[],
    'Regions R': [],
    'Regions G': [],
    'Regions B': [],
    'Fitness R':[],
    'Fitness G':[],
    'Fitness B':[],
    'CPU_time (seconds)': [],
    'MSE':[],
    'RMSE':[],
    'PSNR':[],
    'SSIM':[],
    'UQI':[],
}
for image_name, image_array in rgb_misc_dataset.items():
    # split citra to r,g,b component
    r,g,b = cv2.split(image_array)
    
    for threshold in thresholds:
        for fitness_function in fitness_functions:
            for obj in objs:
                gwo_evaluation_results['image_name'].append(image_name)
                gwo_evaluation_results['thresholds'].append(threshold)
                gwo_evaluation_results['fitness_function'].append(fitness_function)
                gwo_evaluation_results['obj'].append(obj)
                print('\t','='*30)
                print("\t",image_name, "| Threshold = ", threshold)
                print('\t','='*30)
                
                # untuk setiap algorithm run sebanyak n_runs lalu hitung rata-rata hasilnya
                list_cpu_time = []
                list_best_thresholds_r = []
                list_best_thresholds_g = []
                list_best_thresholds_b = []
                list_fitness_r = []
                list_fitness_g = []
                list_fitness_b = []
                list_mse = []
                list_rmse = []
                list_psnr = []
                list_ssim = []
                list_uqi = []
                for i_run in range(n_runs):
                    # PSO optimization
                    gwo_r = EelGrouperOptimizer(
                        k=threshold,
                        eelSize=optimal_params['eelSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_g = EelGrouperOptimizer(
                        k=threshold,
                        eelSize=optimal_params['eelSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    gwo_b = EelGrouperOptimizer(
                        k=threshold,
                        eelSize=optimal_params['eelSize'],
                        maxIteration=optimal_params['maxIteration'],
                        fitness_function=fitness_function,
                        obj='max'
                    )
                    # Evaluation Metrics
                    st = time.time()
                    greyWolfs_b, best_thresholds_b = gwo_r.fit_run(b)
                    greyWolfs_g, best_thresholds_g = gwo_g.fit_run(g)
                    greyWolfs_r, best_thresholds_r = gwo_b.fit_run(r)
                    et = time.time()
                    print(gwo_r.get_params_training_()['best_fitness_tracking'][-1], " | ", gwo_g.get_params_training_()['best_fitness_tracking'][-1]," | ", gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    regions_b = Utilization().digitize(b, best_thresholds_b)
                    regions_g = Utilization().digitize(g, best_thresholds_g)
                    regions_r = Utilization().digitize(r, best_thresholds_r)
                    merge_regions_RGB = cv2.merge((regions_r,regions_g,regions_b))
                    
                    list_cpu_time.append(et-st)
                    list_best_thresholds_r.append(best_thresholds_r)
                    list_best_thresholds_g.append(best_thresholds_g)
                    list_best_thresholds_b.append(best_thresholds_b)
                    list_fitness_r.append(gwo_r.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_g.append(gwo_g.get_params_training_()['best_fitness_tracking'][-1])
                    list_fitness_b.append(gwo_b.get_params_training_()['best_fitness_tracking'][-1])
                    list_mse.append(mse(image_array, merge_regions_RGB))
                    list_rmse.append(rmse(image_array, merge_regions_RGB))
                    list_psnr.append(psnr(image_array, merge_regions_RGB))
                    list_ssim.append(ssim(image_array, merge_regions_RGB)[0])
                    list_uqi.append(uqi(image_array, merge_regions_RGB))
                
                mean_best_thresholds_r = np.round(np.array(list_best_thresholds_r).mean(axis=0)).astype('int')
                mean_best_thresholds_g = np.round(np.array(list_best_thresholds_g).mean(axis=0)).astype('int')
                mean_best_thresholds_b = np.round(np.array(list_best_thresholds_b).mean(axis=0)).astype('int')
                mean_regions_r=Utilization().digitize(r, mean_best_thresholds_r)
                mean_regions_g=Utilization().digitize(g, mean_best_thresholds_g)
                mean_regions_b=Utilization().digitize(b, mean_best_thresholds_b)
                gwo_evaluation_results['Mean best_thresholds R'].append(mean_best_thresholds_r)
                gwo_evaluation_results['Mean best_thresholds G'].append(mean_best_thresholds_g)
                gwo_evaluation_results['Mean best_thresholds B'].append(mean_best_thresholds_b)
                gwo_evaluation_results['Mean Fitness R'].append(np.mean(np.array(list_fitness_r)))
                gwo_evaluation_results['Mean Fitness G'].append(np.mean(np.array(list_fitness_g)))
                gwo_evaluation_results['Mean Fitness B'].append(np.mean(np.array(list_fitness_b)))
                gwo_evaluation_results['Mean CPU_time (seconds)'].append(np.mean(np.array(list_cpu_time)))
                gwo_evaluation_results['Mean MSE'].append(np.mean(np.array(list_mse)))
                gwo_evaluation_results['Mean RMSE'].append(np.mean(np.array(list_rmse)))
                gwo_evaluation_results['Mean PSNR'].append(np.mean(np.array(list_psnr)))
                gwo_evaluation_results['Mean SSIM'].append(np.mean(np.array(list_ssim)))
                gwo_evaluation_results['Mean UQI'].append(np.mean(np.array(list_uqi)))
                gwo_evaluation_results['Regions R'].append(mean_regions_r)
                gwo_evaluation_results['Regions G'].append(mean_regions_g)
                gwo_evaluation_results['Regions B'].append(mean_regions_b)
                gwo_evaluation_results['Fitness R'].append(list_fitness_r)
                gwo_evaluation_results['Fitness G'].append(list_fitness_g)
                gwo_evaluation_results['Fitness B'].append(list_fitness_b)
                gwo_evaluation_results['CPU_time (seconds)'].append(list_cpu_time)
                gwo_evaluation_results['MSE'].append(list_mse)
                gwo_evaluation_results['RMSE'].append(list_rmse)
                gwo_evaluation_results['PSNR'].append(list_psnr)
                gwo_evaluation_results['SSIM'].append(list_ssim)
                gwo_evaluation_results['UQI'].append(list_uqi)
                print(f"Mean Fitness = {np.mean([np.mean(list_fitness_r),np.mean(list_fitness_g),np.mean(list_fitness_b)])}")

	 AirplaneF16.tiff | Threshold =  2
610.7450930005004  |  2220.3503854808123  |  1787.8045911666616
Mean Fitness = 1539.6333565493248
	 AirplaneF16.tiff | Threshold =  3
924.5744040244973  |  2487.8297507773764  |  1808.4165353498536
Mean Fitness = 1740.273563383909
	 AirplaneF16.tiff | Threshold =  4
952.0226877071209  |  2550.5758022075092  |  1847.9970368710083
Mean Fitness = 1783.5318422618795
	 AirplaneF16.tiff | Threshold =  5
962.5886381806777  |  2483.440708120076  |  1825.3593915833583
Mean Fitness = 1757.129579294704
	 Lena.png | Threshold =  2
851.4230574431595  |  2270.1406279056873  |  2039.1399497383895
Mean Fitness = 1720.2345450290788
	 Lena.png | Threshold =  3
913.2747577627117  |  2261.368234350384  |  2166.1878090544665
Mean Fitness = 1780.2769337225207
	 Lena.png | Threshold =  4
1032.2317758161078  |  2503.291790054488  |  2199.490254800933
Mean Fitness = 1911.6712735571764
	 Lena.png | Threshold =  5
1082.434594918552  |  2434.5102278768604  |  2165.0733410565217

## Visualize Results using DataFrame

In [8]:
gwo_evaluation_results_df = pd.DataFrame(gwo_evaluation_results)
gwo_evaluation_results_df

,image_name,thresholds,fitness_function,obj,Mean best_thresholds R,Mean best_thresholds G,Mean best_thresholds B,Mean Fitness R,Mean Fitness G,Mean Fitness B,...,Regions B,Fitness R,Fitness G,Fitness B,CPU_time (seconds),MSE,RMSE,PSNR,SSIM,UQI
0,AirplaneF16.tiff,2,otsu,max,"[120, 178]","[71, 123]","[92, 131]",610.745093,2220.350385,1787.804591,...,"[[131, 255, 255, 255, 255, 255, 255, 255, 255,...",[610.7450930005004],[2220.3503854808123],[1787.8045911666616],[1.1201517581939697],[3041.7120310465493],[55.151718296409854],[13.299622653019288],[0.7156384889749404],[0.9283992211233913]
1,AirplaneF16.tiff,3,otsu,max,"[81, 125, 160]","[69, 101, 159]","[84, 129, 171]",924.574404,2487.829751,1808.416535,...,"[[129, 255, 255, 255, 255, 255, 255, 255, 255,...",[924.5744040244973],[2487.8297507773764],[1808.4165353498536],[0.9697551727294922],[2338.5809936523438],[48.35887709254986],[14.441279451493],[0.7794650423246557],[0.9453753287954504]
2,AirplaneF16.tiff,4,otsu,max,"[66, 104, 133, 168]","[80, 100, 138, 179]","[99, 111, 162, 179]",952.022688,2550.575802,1847.997037,...,"[[99, 255, 255, 255, 255, 255, 255, 255, 255, ...",[952.0226877071209],[2550.5758022075092],[1847.9970368710083],[1.2177324295043945],[2074.472625732422],[45.546378843245286],[14.96172652516027],[0.803733161254681],[0.9525567423683835]
3,AirplaneF16.tiff,5,otsu,max,"[34, 66, 122, 150, 164]","[43, 44, 93, 123, 151]","[90, 109, 143, 159, 177]",962.588638,2483.440708,1825.359392,...,"[[109, 255, 255, 255, 255, 255, 255, 255, 255,...",[962.5886381806777],[2483.440708120076],[1825.3593915833583],[0.9745466709136963],[2239.4833958943686],[47.32318032311827],[14.629325140462058],[0.7889319370486193],[0.9501265204676662]
4,Lena.png,2,otsu,max,"[110, 163]","[58, 106]","[101, 112]",851.423057,2270.140628,2039.139950,...,"[[255, 255, 255, 255, 255, 255, 255, 255, 255,...",[851.4230574431595],[2270.1406279056873],[2039.1399497383895],[0.9645130634307861],[4557.375801086426],[67.50833875223435],[11.543655188975166],[0.567552557415035],[0.8456106336024747]
5,Lena.png,3,otsu,max,"[92, 139, 179]","[47, 72, 100]","[56, 75, 120]",913.274758,2261.368234,2166.187809,...,"[[255, 255, 255, 255, 255, 120, 255, 255, 255,...",[913.2747577627117],[2261.368234350384],[2166.1878090544665],[1.0998218059539795],[4059.520404815674],[63.71436576483889],[12.046056321786796],[0.581837575500853],[0.8697749317277189]
6,Lena.png,4,otsu,max,"[93, 139, 141, 185]","[61, 88, 96, 130]","[65, 77, 95, 129]",1032.231776,2503.291790,2199.490255,...,"[[129, 129, 255, 129, 129, 129, 129, 129, 129,...",[1032.2317758161078],[2503.291790054488],[2199.490254800933],[1.0751266479492188],[2212.013151804606],[47.03204388291674],[14.682926560746768],[0.6491064698185941],[0.9135214348283524]
7,Lena.png,5,otsu,max,"[101, 102, 134, 153, 175]","[39, 65, 81, 92, 115]","[41, 66, 89, 113, 140]",1082.434595,2434.510228,2165.073341,...,"[[140, 140, 140, 140, 140, 140, 140, 140, 140,...",[1082.434594918552],[2434.5102278768604],[2165.0733410565217],[1.043701410293579],[2493.23836517334],[49.93233787009517],[14.163165598086726],[0.6924747905796657],[0.9227252234662459]


# Save results

In [9]:
# pickle.dump(gwo_evaluation_results_df, open('results/evaluation/memGWO/memGWO_otsu_RGB_df.pkl', 'wb'))
# pickle.dump(gwo_evaluation_results, open('results/evaluation/memGWO/memGWO_otsu_RGB_dict.pkl', 'wb'))